## Sort the model first

In [1]:
import os

os.environ['YOLO_VERBOSE'] = 'False'

In [2]:
import torch
from ultralytics import YOLO
from torch import nn

In [3]:
from ultralytics.engine.trainer import BaseTrainer
from ultralytics.utils import yaml_load, LOGGER, RANK, DEFAULT_CFG_DICT, DEFAULT_CFG_KEYS
from ultralytics.utils.checks import check_yaml
from ultralytics.engine.model import Model
from ultralytics.utils.torch_utils import initialize_weights, de_parallel
from ultralytics.nn.tasks import attempt_load_one_weight
from copy import deepcopy
from datetime import datetime
from pathlib import Path
from typing import List, Union

In [4]:
from utils import model_sorting, utils
from channel_pruning import pruner

In [19]:
model = YOLO('tsr.pt')

In [20]:
def save_model_v2(self: BaseTrainer):
    """
    Disabled half precision saving. originated from ultralytics/yolo/engine/trainer.py
    """
    ckpt = {
        'epoch': self.epoch,
        'best_fitness': self.best_fitness,
        'model': deepcopy(de_parallel(self.model)),
        'ema': deepcopy(self.ema.ema),
        'updates': self.ema.updates,
        'optimizer': self.optimizer.state_dict(),
        'train_args': vars(self.args),  # save as dict
        'date': datetime.now().isoformat(),
        'version': __version__}

    # Save last, best and delete
    torch.save(ckpt, self.last)
    if self.best_fitness == self.fitness:
        torch.save(ckpt, self.best)
    if (self.epoch > 0) and (self.save_period > 0) and (self.epoch % self.save_period == 0):
        torch.save(ckpt, self.wdir / f'epoch{self.epoch}.pt')
    del ckpt


def final_eval_v2(self: BaseTrainer):
    """
    originated from ultralytics/yolo/engine/trainer.py
    """
    for f in self.last, self.best:
        if f.exists():
            strip_optimizer_v2(f)  # strip optimizers
            if f is self.best:
                LOGGER.info(f'\nValidating {f}...')
                self.metrics = self.validator(model=f)
                self.metrics.pop('fitness', None)
                self.run_callbacks('on_fit_epoch_end')


def strip_optimizer_v2(f: Union[str, Path] = 'best.pt', s: str = '') -> None:
    """
    Disabled half precision saving. originated from ultralytics/yolo/utils/torch_utils.py
    """
    x = torch.load(f, map_location=torch.device('cpu'))
    args = {**DEFAULT_CFG_DICT, **x['train_args']}  # combine model args with default args, preferring model args
    if x.get('ema'):
        x['model'] = x['ema']  # replace model with ema
    for k in 'optimizer', 'ema', 'updates':  # keys
        x[k] = None
    for p in x['model'].parameters():
        p.requires_grad = False
    x['train_args'] = {k: v for k, v in args.items() if k in DEFAULT_CFG_KEYS}  # strip non-default keys
    # x['model'].args = x['train_args']
    torch.save(x, s or f)
    mb = os.path.getsize(s or f) / 1E6  # filesize
    LOGGER.info(f"Optimizer stripped from {f},{f' saved as {s},' if s else ''} {mb:.1f}MB")



In [21]:
def train_v2(self: YOLO,trainer=None, pruning=False, **kwargs):
    """
    Disabled loading new model when pruning flag is set. originated from ultralytics/yolo/engine/model.py
    """

    self._check_is_pytorch_model()
    if self.session:  # Ultralytics HUB session
        if any(kwargs):
            LOGGER.warning('WARNING ⚠️ using HUB training arguments, ignoring local training arguments.')
        kwargs = self.session.train_args
    overrides = self.overrides.copy()
    overrides.update(kwargs)
    if kwargs.get('cfg'):
        LOGGER.info(f"cfg file passed. Overriding default params with {kwargs['cfg']}.")
        overrides = yaml_load(check_yaml(kwargs['cfg']))
    overrides['mode'] = 'train'
    if not overrides.get('data'):
        raise AttributeError("Dataset required but missing, i.e. pass 'data=coco128.yaml'")
    if overrides.get('resume'):
        overrides['resume'] = self.ckpt_path

    # self.task = overrides.get('task') or self.task
    # self.trainer = Model.task_map[self.task][1](overrides=overrides, _callbacks=self.callbacks)
    self.trainer = (trainer or self._smart_load("trainer"))(overrides=overrides, _callbacks=self.callbacks)

    if not pruning:
        if not overrides.get('resume'):  # manually set model only if not resuming
            self.trainer.model = self.trainer.get_model(weights=self.model if self.ckpt else None, cfg=self.model.yaml)
            self.model = self.trainer.model

    else:
        # pruning mode
        self.trainer.pruning = True
        self.trainer.model = self.model

        # replace some functions to disable half precision saving
        self.trainer.save_model = save_model_v2.__get__(self.trainer)
        self.trainer.final_eval = final_eval_v2.__get__(self.trainer)

    self.trainer.hub_session = self.session  # attach optional HUB session
    self.trainer.train()
    # Update model and cfg after training
    if RANK in (-1, 0):
        self.model, _ = attempt_load_one_weight(str(self.trainer.best))
        self.overrides = self.model.args
        self.metrics = getattr(self.trainer.validator, 'metrics', None)

In [22]:
pruning_cfg = yaml_load(check_yaml('default.yaml'))

In [23]:
pruning_cfg['device'] = '0' 

In [24]:
pruning_cfg['data'] = "/home/dell/Yolov8_Optimization/road_sign_data.yaml"
pruning_cfg['epochs'] = 1

In [25]:
pruning_cfg['name'] = "finetune"

In [26]:
model.__setattr__("train_v2", train_v2.__get__(model))

In [27]:
sorted_model = model_sorting.sort_model(model)

In [28]:
sorted_model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## Pruning

In [29]:
pruned_model=pruner.prune_layer(sorted_model,p_ratio=0.3)        

Conv Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
SPPF Weight transfer complete.
None
C2f Weight transfer complete.
135
C2f Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
Conv Weight transfer complete.
C2f Weight transfer complete.
Sequential weight transfer complete.


In [30]:
pruned_model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 11, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (1): Conv(
        (conv): Conv2d(11, 22, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(22, 22, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (cv2): Conv(
          (conv): Conv2d(33, 22, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
     

In [31]:
@torch.inference_mode()
def evaluate(
  model: nn.Module,
) -> float:
  
  metrics = model.val(data = 'road_sign_data.yaml', split = 'val')
  acc = metrics.results_dict['metrics/mAP50(B)']

  return acc

In [32]:
model.train_v2(pruning=True, **pruning_cfg)

/home/dell/Yolov8_Optimization/venv/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return F.conv2d(input, weight, bias, self.stride,


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 

In [ ]:
# model_accuracy = evaluate(pruned_model)

In [ ]:
# def fine_tune(model: nn.Module):
#   model.train(data='road_sign_data.yaml', epochs=10, imgsz=640,device = '0')

In [ ]:
# fine_tune(pruned_model)

In [ ]:
# utils.get_model_size(pruned_model)

In [ ]:
# utils.get_model_size(model)